In [81]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
import tabulate 

from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


In [6]:
user_train = pd.read_csv('user_train.csv')
item_train = pd.read_csv('item_train.csv')

In [7]:
user_train.head(10)

,user_id,overall_score,Action,Adult,Adventure,Anime,Comedy,Crime,DIY,Drama,...,Mystery,Psychological,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,Youth
0,0,9.0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
1,0,6.5,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
2,0,9.0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
3,0,7.0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
4,0,8.5,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
5,0,9.0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
6,0,8.0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
7,0,8.5,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
8,0,8.5,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
9,0,6.5,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0


In [8]:
item_train.head(10)

,drama_name,kdrama_id,pop,rank,Action,Adult,Adventure,Anime,Comedy,Crime,...,Mystery,Psychological,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,Youth
0,Sing My Crush,1.0,2238.0,1484.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Happy Merry Ending,32.0,1366.0,7632.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Our Dating Sim,51.0,569.0,2185.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,The Director Who Buys Me Dinner,80.0,679.0,6529.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Unlock My Boss,84.0,1088.0,2117.0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
5,Roommates of Poongduck 304,114.0,570.0,2870.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,The Golden Spoon,127.0,658.0,1903.0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
7,Big Mouth,150.0,163.0,395.0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
8,Blueming,203.0,367.0,1715.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,Grid,223.0,1195.0,5507.0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


<h4> Switch kdrama_id and drama_name Columns </h4>

In [9]:
cols = item_train.columns.tolist()
cols[0], cols[1] = cols[1], cols[0]
item_train = item_train[cols]

In [10]:
item_train.head(10)

,kdrama_id,drama_name,pop,rank,Action,Adult,Adventure,Anime,Comedy,Crime,...,Mystery,Psychological,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,Youth
0,1.0,Sing My Crush,2238.0,1484.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,32.0,Happy Merry Ending,1366.0,7632.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,51.0,Our Dating Sim,569.0,2185.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,80.0,The Director Who Buys Me Dinner,679.0,6529.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,84.0,Unlock My Boss,1088.0,2117.0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
5,114.0,Roommates of Poongduck 304,570.0,2870.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,127.0,The Golden Spoon,658.0,1903.0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
7,150.0,Big Mouth,163.0,395.0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
8,203.0,Blueming,367.0,1715.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,223.0,Grid,1195.0,5507.0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [11]:
y_train = user_train['overall_score']


In [12]:
type(y_train)


pandas.core.series.Series

In [13]:

item = item_train
item_train = item_train.drop(['kdrama_id', 'drama_name'],axis = 1)

In [14]:
user = user_train
user_train = user_train.drop(['user_id', 'overall_score'], axis =1)

<h4> Scale training Data </h4>

In [15]:

item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.values.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.values.reshape(-1, 1))




In [16]:
print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [17]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (7798, 34)
movie/item test data shape: (1950, 34)


In [18]:
num_user_features = user_train.shape[1]
num_item_features = item_train.shape[1]

In [19]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(num_outputs, activation = 'linear'),
  
  
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(num_outputs, activation = 'linear'),
  
  
])

input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

output = tf.keras.layers.Dot(axes=1)([vu, vm])

model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 34)]                 0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   45472     ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   45984     ['input_2[0][0]']             
                                                                                              

In [20]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [21]:
tf.random.set_seed(1)
model.fit([user_train, item_train], y_train, epochs=100)

Epoch 1/100
244/244 [==============================] - 0s 1ms/step - loss: 0.1204
Epoch 2/100
244/244 [==============================] - 0s 1ms/step - loss: 0.0933
Epoch 3/100
244/244 [==============================] - 0s 1ms/step - loss: 0.0819
Epoch 4/100
244/244 [==============================] - 0s 1ms/step - loss: 0.0778
Epoch 5/100
244/244 [==============================] - 0s 1ms/step - loss: 0.0732
Epoch 6/100
244/244 [==============================] - 0s 1ms/step - loss: 0.0690
Epoch 7/100
244/244 [==============================] - 0s 993us/step - loss: 0.0663
Epoch 8/100
244/244 [==============================] - 0s 989us/step - loss: 0.0647
Epoch 9/100
244/244 [==============================] - 0s 1ms/step - loss: 0.0622
Epoch 10/100
244/244 [==============================] - 0s 969us/step - loss: 0.0604
Epoch 11/100
244/244 [==============================] - 0s 1ms/step - loss: 0.0591
Epoch 12/100
244/244 [==============================] - 0s 1ms/step - loss: 0.0575
Epoch 1

In [22]:
model.evaluate([user_test, item_test], y_test)

61/61 [==============================] - 0s 445us/step - loss: 0.0899


0.08991076052188873

<h2> #1 PREDCIT RATINGS FOR USERS THAT HAVE ALREADY RATED THE DRAMAS AND COMPARE </h2>

In [23]:
user.head()

,user_id,overall_score,Action,Adult,Adventure,Anime,Comedy,Crime,DIY,Drama,...,Mystery,Psychological,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,Youth
0,0,9.0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
1,0,6.5,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
2,0,9.0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
3,0,7.0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
4,0,8.5,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0


In [24]:
uid = 0
user_vecs = user[user['user_id']== uid]

y_vecs = user_vecs['overall_score']
user_vecs = user_vecs.drop(['user_id', 'overall_score'], axis = 1)
y_vecs.head(30)

0     9.0
1     6.5
2     9.0
3     7.0
4     8.5
5     9.0
6     8.0
7     8.5
8     8.5
9     6.5
10    9.0
11    9.0
12    6.5
13    8.5
14    8.0
15    8.0
16    7.0
17    8.0
18    6.5
19    8.0
20    9.0
21    8.0
Name: overall_score, dtype: float64

In [25]:
user_vecs.head(30)

,Action,Adult,Adventure,Anime,Comedy,Crime,DIY,Drama,Espionage,Family,...,Mystery,Psychological,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,Youth
0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,0.0,0.0,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
1,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,0.0,0.0,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
2,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,0.0,0.0,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
3,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,0.0,0.0,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
4,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,0.0,0.0,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
5,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,0.0,0.0,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
6,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,0.0,0.0,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
7,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,0.0,0.0,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
8,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,0.0,0.0,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
9,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,0.0,0.0,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0


In [26]:
item.head()

,kdrama_id,drama_name,pop,rank,Action,Adult,Adventure,Anime,Comedy,Crime,...,Mystery,Psychological,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,Youth
0,1.0,Sing My Crush,2238.0,1484.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,32.0,Happy Merry Ending,1366.0,7632.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,51.0,Our Dating Sim,569.0,2185.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,80.0,The Director Who Buys Me Dinner,679.0,6529.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,84.0,Unlock My Boss,1088.0,2117.0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [27]:
index_u = user_vecs.index.tolist()
item_vecs = item.iloc[index_u[0]:index_u[-1]+1,2:]
item_vecs.head(23)

,pop,rank,Action,Adult,Adventure,Anime,Comedy,Crime,DIY,Drama,...,Mystery,Psychological,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,Youth
0,2238.0,1484.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1366.0,7632.0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,569.0,2185.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,679.0,6529.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1088.0,2117.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,570.0,2870.0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
6,658.0,1903.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
7,163.0,395.0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
8,367.0,1715.0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
9,1195.0,5507.0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0


In [28]:





# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs, sitem_vecs])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)


#

1/1 [==============================] - 0s 40ms/step


In [29]:
print(y_pu)

[[9.018528 ]
 [6.939753 ]
 [8.98087  ]
 [7.7372875]
 [8.788444 ]
 [8.2964115]
 [8.125534 ]
 [9.2931185]
 [8.640123 ]
 [6.5876303]
 [8.898846 ]
 [8.461033 ]
 [6.849948 ]
 [8.194017 ]
 [8.066369 ]
 [8.384446 ]
 [6.911785 ]
 [9.111548 ]
 [6.530284 ]
 [7.49252  ]
 [8.630753 ]
 [7.487232 ]]


In [30]:
# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  
sorted_ypu   = y_pu[sorted_index]
v = item[['kdrama_id','drama_name']].iloc[index_u[0]:index_u[-1]+1,:]
sorted_items = v.reset_index(drop= True).iloc[sorted_index]
sorted_user  = user.iloc[sorted_index]


sorted_y = y_vecs.reset_index(drop = True).iloc[sorted_index]


In [31]:
type(sorted_ypu)
sorted_ypred = pd.DataFrame(sorted_ypu)
sorted_ypred.head(10)

,0
0,9.293118
1,9.111548
2,9.018528
3,8.980870
4,8.898846
5,8.788444
6,8.640123
7,8.630753
8,8.461033
9,8.384446


In [32]:

out = pd.concat([sorted_items, sorted_y], axis=1)
out['prediction_score'] = sorted_ypred.values

<h3> ACTUAL RATING V/S PREDICTED RATING </h3>


In [33]:
out.head(30)

,kdrama_id,drama_name,overall_score,prediction_score
7,150.0,Big Mouth,8.5,9.293118
17,300.0,Dali and the Cocky Prince,8.0,9.111548
0,1.0,Sing My Crush,9.0,9.018528
2,51.0,Our Dating Sim,9.0,8.980870
10,224.0,Semantic Error,9.0,8.898846
4,84.0,Unlock My Boss,8.5,8.788444
8,203.0,Blueming,8.5,8.640123
20,496.0,Where Your Eyes Linger,9.0,8.630753
11,225.0,Twenty-Five Twenty-One,9.0,8.461033
15,276.0,Happiness,8.0,8.384446


<h2> #2 PREDICT DRAMA RATINGS FOR A NEW USER 
    SORTED by RATINGS (HIGHEST TO LOWEST) BASED ON WHICH GENRE HE/SHE PREFERS</h2>

In [34]:
drama_vecs = item.drop_duplicates(subset=['drama_name'])

In [35]:
drama_vecs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1104 entries, 0 to 9570
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   kdrama_id        1104 non-null   float64
 1   drama_name       1104 non-null   object 
 2   pop              1104 non-null   float64
 3   rank             1104 non-null   float64
 4   Action           1104 non-null   int64  
 5   Adult            1104 non-null   int64  
 6   Adventure        1104 non-null   int64  
 7   Anime            1104 non-null   int64  
 8   Comedy           1104 non-null   int64  
 9   Crime            1104 non-null   int64  
 10  DIY              1104 non-null   int64  
 11  Drama            1104 non-null   int64  
 12  Espionage        1104 non-null   int64  
 13  Family           1104 non-null   int64  
 14  Fantasy          1104 non-null   int64  
 15  Food             1104 non-null   int64  
 16  Friendship       1104 non-null   int64  
 17  Historical       11

<h4>Create New User Vector</h4>

In [36]:
user.head()

,user_id,overall_score,Action,Adult,Adventure,Anime,Comedy,Crime,DIY,Drama,...,Mystery,Psychological,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,Youth
0,0,9.0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
1,0,6.5,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
2,0,9.0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
3,0,7.0,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0
4,0,8.5,8.0,0.0,0.0,0.0,8.333333,0.0,0.0,7.807692,...,6.5,0.0,8.269231,0.0,0.0,8.0,7.5,0.0,0.0,0.0


In [37]:
new_user_vecs = user.iloc[0:1]

In [38]:
new_user_vecs = new_user_vecs*0

In [39]:
new_user_vecs.head()

,user_id,overall_score,Action,Adult,Adventure,Anime,Comedy,Crime,DIY,Drama,...,Mystery,Psychological,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,Youth
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h4>Inititalize which genre the new user prefers.</h4>

In [40]:
new_user_vecs['Romance'] = 10
new_user_vecs['Comedy'] = 9



In [41]:
row_array = new_user_vecs.values
repeated_array = np.tile(row_array, (len(drama_vecs), 1))

In [42]:
new_user_vecs = pd.DataFrame(
    repeated_array, 
    columns=new_user_vecs.columns
)


In [43]:
new_user_vecs = new_user_vecs.drop(['user_id', 'overall_score'], axis = 1)

<h4> Scale and Predict</h4>

In [44]:
snuser_vecs = scalerUser.transform(new_user_vecs)
snitem_vecs = scalerItem.transform(drama_vecs.iloc[:,2:])

In [45]:
y_p = model.predict([snuser_vecs, snitem_vecs])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

35/35 [==============================] - 0s 407us/step


In [46]:

sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  
sorted_ypu   = y_pu[sorted_index]
sorted_ypu = pd.DataFrame(sorted_ypu)
sorted_items = drama_vecs.iloc[sorted_index] 

In [47]:
result = sorted_items.iloc[:,1:4]

In [48]:
result['predicted_ratings'] = sorted_ypu.values

<h4> Import CSV with drama name and genre</h4>

In [95]:
drama_with_genre = pd.read_csv('data_preprocessing/korean_drama_complete.csv')

In [96]:
drama_with_genre.head()

,kdrama_id,drama_name,year,director,screenwriter,country,type,tot_eps,duration,start_dt,end_dt,aired_on,org_net,content_rt,synopsis,rank,pop,Genre,Genre_Source
0,661d4193916c4e71a2c70473ab11e9e8,Sing My Crush,2023,['So Joon Moon'],NaN,South Korea,Drama,8,1500.0,"Aug 2, 2023","Aug 2, 2023",Wednesday,NaN,Not Yet Rated,Follow the story of acquaintances Ba Ram and H...,1484,2238,"Romance, Music",Predefined
1,5ffcbeaa17114714af1959129984274c,D.P. Season 2,2023,NaN,['Kim Bo Tong'],South Korea,Drama,6,3000.0,"Jul 28, 2023","Jul 28, 2023",Friday,Netflix,15+ - Teens 15 or older,This unfolding story ensues when military dese...,164,1084,"Action, Drama, Military",Predefined
2,65075cb9c1a54be4a441cee6f16c9fdf,Shadow Detective Season 2,2023,['Han Dong Hwa'],"['Song Jung Woo', 'Hwang Seol Hun']",South Korea,Drama,8,3300.0,2023-07-05,2023-07-26,Wednesday,Disney+ Hulu,15+ - Teens 15 or older,Unfolds the ultimate counterattack of veteran ...,2443,6915,"Crime, Mystery, Thriller",Predefined
3,df0f0ac4b3ff4b15afa26f5a7a53a328,To Be Honest,2023,NaN,NaN,South Korea,Drama,3,600.0,2023-06-30,2023-07-14,Friday,NaN,Not Yet Rated,Don't you have those days where the whole univ...,49895,99999,"Drama, Romance",Predefined
4,04c1fe41948e464fb440001831d74d41,Celebrity,2023,['Kim Chul Gyu'],['Kim Yi Young'],South Korea,Drama,12,2700.0,"Jun 30, 2023","Jun 30, 2023",Friday,Netflix,18+ Restricted (violence & profanity),Fame. Money. Power. One young woman fights to ...,826,547,"Drama, Thriller",Predefined


In [98]:
merged_result = pd.merge(
    result, 
    drama_with_genre[['drama_name', 'Genre']], 
    on='drama_name', 
    how='left'
)

In [100]:
merged_result = merged_result[['drama_name', 'Genre', 'pop', 'rank', 'predicted_ratings']]

<h3> RATINGS PREDICTION FOR A NEW USER</h3>

In [103]:
merged_result.head(50)

,drama_name,Genre,pop,rank,predicted_ratings
0,The Road: The Tragedy of One,"Comedy, Fantasy, Romance",4067.0,6756.0,9.808828
1,The New Employee,"Comedy, Romance",818.0,6343.0,9.715191
2,Strangers Again,"Comedy, Romance",1150.0,6757.0,9.712392
3,Reunited Worlds,"Comedy, Fantasy, Romance",1020.0,4837.0,9.712343
4,Revolutionary Love,"Comedy, Romance",519.0,5940.0,9.700036
5,Falling for Do Jeon,"Comedy, Romance",1561.0,7884.0,9.680270
6,A Business Proposal,"Romance, Comedy",22.0,244.0,9.659593
7,Something About 1 Percent,"Comedy, Romance",204.0,1342.0,9.656398
8,Hometown Cha-Cha-Cha,"Romance, Comedy",42.0,197.0,9.655071
9,Yumi's Cells,"Comedy, Romance",245.0,650.0,9.648402


<h2> #3 SHOW SIMILAR DRAMAS </h2>

In [50]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        
model_m = tf.keras.Model(input_item_m, vm_m)                                
model_m.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 34)]              0         
                                                                 
 sequential_1 (Sequential)   (None, 32)                45984     
                                                                 
 tf.math.l2_normalize_2 (TF  (None, 32)                0         
 OpLambda)                                                       
                                                                 
Total params: 45984 (179.62 KB)
Trainable params: 45984 (179.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [67]:
drama_vecs.head()

,kdrama_id,drama_name,pop,rank,Action,Adult,Adventure,Anime,Comedy,Crime,...,Mystery,Psychological,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,Youth
0,1.0,Sing My Crush,2238.0,1484.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,32.0,Happy Merry Ending,1366.0,7632.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,51.0,Our Dating Sim,569.0,2185.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,80.0,The Director Who Buys Me Dinner,679.0,6529.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,84.0,Unlock My Boss,1088.0,2117.0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [55]:
scaled_item_vecs = scalerItem.transform(drama_vecs.iloc[:,2:])
vms = model_m.predict(scaled_item_vecs)
print(f"size of all predicted movie feature vectors: {vms.shape}")

35/35 [==============================] - 0s 414us/step
size of all predicted movie feature vectors: (1104, 32)


In [92]:
drama_with_genre.head()

,kdrama_id,drama_name,year,director,screenwriter,country,type,tot_eps,duration,start_dt,end_dt,aired_on,org_net,content_rt,synopsis,rank,pop,Genre,Genre_Source
0,661d4193916c4e71a2c70473ab11e9e8,Sing My Crush,2023,['So Joon Moon'],NaN,South Korea,Drama,8,1500.0,"Aug 2, 2023","Aug 2, 2023",Wednesday,NaN,Not Yet Rated,Follow the story of acquaintances Ba Ram and H...,1484,2238,"Romance, Music",Predefined
1,5ffcbeaa17114714af1959129984274c,D.P. Season 2,2023,NaN,['Kim Bo Tong'],South Korea,Drama,6,3000.0,"Jul 28, 2023","Jul 28, 2023",Friday,Netflix,15+ - Teens 15 or older,This unfolding story ensues when military dese...,164,1084,"Action, Drama, Military",Predefined
2,65075cb9c1a54be4a441cee6f16c9fdf,Shadow Detective Season 2,2023,['Han Dong Hwa'],"['Song Jung Woo', 'Hwang Seol Hun']",South Korea,Drama,8,3300.0,2023-07-05,2023-07-26,Wednesday,Disney+ Hulu,15+ - Teens 15 or older,Unfolds the ultimate counterattack of veteran ...,2443,6915,"Crime, Mystery, Thriller",Predefined
3,df0f0ac4b3ff4b15afa26f5a7a53a328,To Be Honest,2023,NaN,NaN,South Korea,Drama,3,600.0,2023-06-30,2023-07-14,Friday,NaN,Not Yet Rated,Don't you have those days where the whole univ...,49895,99999,"Drama, Romance",Predefined
4,04c1fe41948e464fb440001831d74d41,Celebrity,2023,['Kim Chul Gyu'],['Kim Yi Young'],South Korea,Drama,12,2700.0,"Jun 30, 2023","Jun 30, 2023",Friday,Netflix,18+ Restricted (violence & profanity),Fame. Money. Power. One young woman fights to ...,826,547,"Drama, Thriller",Predefined


<h4>DRAMAS SIMILAR TO THE TOP 50 DRAMAS SORTED BY POPULARITY</h4>

In [109]:
sorted_indices = drama_vecs['pop'].argsort()


vms = vms[sorted_indices]
drama_vecs = drama_vecs.iloc[sorted_indices].reset_index(drop=True)

count = 50  # number of dramas to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = np.sum((vms[i,:] - vms [j, :])**2)
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genre1", "movie2", "genre2"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    drama1_id = int(drama_vecs.iloc[i,0])
    drama2_id = int(drama_vecs.iloc[min_idx,0])
    name1 = drama_vecs[drama_vecs['kdrama_id'] == drama1_id]['drama_name'].iloc[0]
    name2 = drama_vecs[drama_vecs['kdrama_id'] == drama2_id]['drama_name'].iloc[0]
    
    genre1 = drama_with_genre[drama_with_genre['drama_name'] == name1]['Genre'].iloc[0]
    
    genre2 = drama_with_genre[drama_with_genre['drama_name'] == name2]['Genre'].iloc[0]

    disp.append([name1, genre1, name2, genre2])
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
table

movie1,genre1,movie2,genre2
Goblin,"Drama, Fantasy, Romance",It's Okay to Not Be Okay,"Drama, Fantasy, Romance"
Strong Woman Do Bong Soon,"Comedy, Action, Fantasy",Poetry Story,"Music, History"
Weightlifting Fairy Kim Bok Joo,"Drama, Romance, Sports",Run On,"Drama, Romance, Sports"
Descendants of the Sun,"Drama, Romance, Medical",Dr. Romantic,"Drama, Romance, Medical"
What's Wrong with Secretary Kim,"Drama, Romance",Fight for My Way,"Drama, Romance"
W,"Crime, Mystery, Thriller",The Smile Has Left Your Eyes,"Crime, Thriller, Mystery"
It's Okay to Not Be Okay,"Drama, Fantasy, Romance",Goblin,"Drama, Fantasy, Romance"
Crash Landing on You,"Romance, Comedy, Drama",Reply 1988,"Drama, Comedy, Romance"
Hotel del Luna,"Drama, Fantasy, Romance",Goblin,"Drama, Fantasy, Romance"
While You Were Sleeping,"Fantasy, Romance, Legal",You I Didn't Know,"Comedy, Romance"
